In [1]:
import sys
import os
import logging
from collections import OrderedDict

import numpy as np

import pygpc
from pygpc import AbstractModel

import h5py

from matplotlib import pyplot as plt

from dagma.linear import DagmaLinear
from dagma.nonlinear import DagmaMLP, DagmaNonlinear

from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import IIDSimulation, DAG
from castle.algorithms import Notears

logging.disable(logging.CRITICAL)

2024-07-23 16:14:35,343 - /opt/homebrew/Caskroom/miniconda/base/envs/causal/lib/python3.12/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2024-07-23 16:14:35,424 - /opt/homebrew/Caskroom/miniconda/base/envs/causal/lib/python3.12/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [2]:
class NoTearsLinearCausalDiscoveryModel(AbstractModel):

    def __init__(self, n_nodes, n_edges, n_samples, method, sem_type):
        self.n_nodes = n_nodes
        self.n_edges = n_edges
        self.n_samples = n_samples
        self.method = method
        self.sem_type = sem_type
        self.model = Notears()
    
    def simulate(self, process_id=None, matlab_engine=None):
        W_res_lst = []
        for noise_scale in self.p["noise_scale"]:
            W_true = DAG.scale_free(n_nodes=self.n_nodes, n_edges=self.n_edges)
            data = IIDSimulation(W=W_true, n=self.n_samples, method=self.method, sem_type=self.sem_type, noise_scale=float(noise_scale))
            self.model.learn(data.X)
            W_res_lst.append((W_true - self.model.causal_matrix).flatten())
        return np.stack(W_res_lst, axis=0)

In [3]:
model = NoTearsLinearCausalDiscoveryModel(10, 20, 100, 'linear', 'gauss')
model.set_parameters({"noise_scale": np.array([1, 2, 3])})
res = model.simulate()

In [5]:
fn_results = os.path.join(os.getcwd(), "data")

## Define model
model = NoTearsLinearCausalDiscoveryModel(10, 20, 100, 'linear', 'gauss')

## Define parameters
parameters = OrderedDict()
parameters["noise_scale"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[0, 100])

# Define problem
problem = pygpc.Problem(model, parameters)

# gPC options
options = OrderedDict()
options["solver"] = "Moore-Penrose"
options["method"] = "reg"
options["order"] = [10]
options["order_max"] = 10    
options["order_max_norm"] =  2.
options["interaction_order"] = 1
options['n_cpu'] = 1
options["fn_results"] = "./results.hdf5"
    
# Define algorithm
algorithm = pygpc.Static(problem=problem, options=options)

# Initialize gPC Session
session = pygpc.Session(algorithm=algorithm)

# Run gPC session
session, coeffs, results = session.run()


Process SpawnPoolWorker-7:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/causal/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/causal/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/causal/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/causal/lib/python3.12/multiprocessing/process.py", line 

KeyboardInterrupt: 

In [6]:
fn_results = os.path.join(os.getcwd(), "data")

## Define model
# model = pygpc.testfunctions.Ishigami()
model = NoTearsLinearCausalDiscoveryModel(10, 20, 100, 'linear', 'gauss')

## Define parameters
parameters = OrderedDict()
parameters["noise_scale"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[0, 100])

# Define problem
problem = pygpc.Problem(model, parameters)

# gPC options
options = dict()
options["solver"] = "LarsLasso"
options["settings"] = None
options["order_start"] = 3
options["order_end"] = 15
options["seed"] = 1
options["projection"] = False
options["order_max_norm"] = 1.
options["interaction_order"] = 2
options["matrix_ratio"] = 2
options["n_cpu"] = 0
options["gradient_enhanced"] = False
options["gradient_calculation"] = "standard_forward"
options["error_type"] = "loocv"
options["error_norm"] = "absolute"
options["n_samples_validation"] = 1e2
options["qoi"] = 0
options["classifier"] = "learning"
options["n_samples_discontinuity"] = 10
options["adaptive_sampling"] = True
options["eps"] = 0.03
options["n_grid_init"] = 100
options["GPU"] = False
options["fn_results"] = fn_results
options["classifier_options"] = {"clusterer": "KMeans",
                                 "n_clusters": 2,
                                 "classifier": "MLPClassifier",
                                 "classifier_solver": "lbfgs"}

# define algorithm
algorithm = pygpc.MERegAdaptiveProjection(problem=problem, options=options)

# Initialize gPC Session
session = pygpc.Session(algorithm=algorithm)

# Run gPC session
session, coeffs, results = session.run()


Creating initial grid (Random) with n_grid=100
Performing 100 initial simulations!


KeyboardInterrupt: 